# Backward Propagation

正向传播 (Forward Propagation) 就是通过输入样本和模型数据得到预测值，反向传播 (Backward Propagation) 就是计算损失函数相对于模型参数的梯度，从而对模型进行优化。  

[ForwardBackwardPropagation](../Images/backward.png)

## PyTorch Implementation

首先导入 `torch`，然后初始化一个张量。

In [43]:
import torch

x = torch.arange(4.0)
print(x)

tensor([0., 1., 2., 3.])


在计算y关于x的梯度之前，**声明梯度是需要的**。

In [44]:
# 等价于 x = torch.arange(4, requires_grad=True)
# 可以在创建张量时就声明需要梯度
x.requires_grad_(True)

tensor([0., 1., 2., 3.], requires_grad=True)

定义：$y = 2 x^2 = 2 x^T x$，现在就可以计算$y$的梯度了。注意：**在计算梯度时 (调用 `backward` 方法)，它只能计算一个标量 (scalar) 的梯度**。如果y是一个标量，可以直接使用 `backward` 方法，如果y不是一个标量，而是一个张量或者矩阵，那么就必须先转化为一个标量 (例如使用 `sum` 求和) ，然后再使用 `backward` 方法计算梯度。 

In [45]:
y = 2 * torch.dot(x, x)
print(y)

tensor(28., grad_fn=<MulBackward0>)


调用反向传播函数 `backward` 来自动计算y关于x在每个样本点上的梯度。

In [46]:
y.backward()
print(x.grad)

# 检验y关于x的导数是不是y'= 4x
print(x.grad == 4 * x)

tensor([ 0.,  4.,  8., 12.])
tensor([True, True, True, True])


在默认情况下，PyTorch会累积梯度。在第一次梯度计算中，得到的是正确的结果；在第二次梯度计算中，得到的是第一次的结果加上第二次的结果。PyTorch采用梯度累积的原因是为了支持某些复杂的场景，在这里暂时省略。

In [47]:
x = torch.arange(4.0, requires_grad=True)

y1 = 2 * torch.dot(x, x)
y1.backward()
print(f'The first computation of gradient: {x.grad}')

y2 = torch.dot(x, x)
y2.backward()
print(f'The second computation of gradient: {x.grad}')

The first computation of gradient: tensor([ 0.,  4.,  8., 12.])
The second computation of gradient: tensor([ 0.,  6., 12., 18.])


如果想要避免出现梯度累积的情况，可以使用 `grad.zero_` 方法。这样得到的梯度就是 $y = 2 x^2, y = x^2$ 下分别的梯度。

In [48]:
y1 = 2 * torch.dot(x, x)
y1.backward()
print(f'The first computation of gradient: {x.grad}')

x.grad.zero_()

y2 = torch.dot(x, x)
y2.backward()
print(f'The second computation of gradient: {x.grad}')

x.grad.zero_()

The first computation of gradient: tensor([ 0., 10., 20., 30.])
The second computation of gradient: tensor([0., 2., 4., 6.])


tensor([0., 0., 0., 0.])

在PyTorch中，[计算图](#backward-propagation) (Computational Graph) 是自动求导的基础，用于表示张量之间的计算关系和操作顺序。PyTorch的自动求导模块通过计算图来追踪和计算张量的梯度。可以通过 `detach` 方法将，某些计算移到计算图之外。

In [49]:
# y是关于x的函数，但是u不是关于x的函数了
y = x ** 2
u = y.detach()

# 如果z关于x求导，得到的就是u
z = u * x
z.sum().backward()

print(x.grad)
print(x.grad == u)

tensor([0., 1., 4., 9.])
tensor([True, True, True, True])
